In [29]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import time
from tqdm import tqdm

In [2]:
def scrape_NBA_draft_data(year):
    # drafts URL
    url = f"https://www.basketball-reference.com/draft/NBA_{year}.html"

    # HTML of the page
    html = urlopen(url)

    # soup
    soup = BeautifulSoup(html, features="lxml")

    # headers
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]

    rows = soup.findAll('tr')[2:]
    draft_data = [[td.getText() for td in rows[i].findAll('td')]
                 for i in range(len(rows))]

    return pd.DataFrame(draft_data, columns=headers[1:])


In [3]:
df_full = pd.DataFrame()


for year in tqdm(range(1950, 2007)):
    df_year = scrape_NBA_draft_data(year)
    # adding year column
    df_year['Year'] = year
    df_full = pd.concat([df_full, df_year], ignore_index=True)
    time.sleep(6)



100%|██████████| 57/57 [06:03<00:00,  6.38s/it]


In [4]:
df_full

,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,AST,...,FT%,MP,PTS,TRB,AST,WS,WS/48,BPM,VORP,Year
0,3,PHW,Paul Arizin,Villanova,10,713,24897,16266,6129,1665,...,.810,38.4,22.8,8.6,2.3,108.8,.183,,,1950
1,1,BOS,Chuck Share,Bowling Green,9,596,13023,4928,4986,809,...,.693,21.9,8.3,8.4,1.4,42.0,.155,,,1950
2,2,BLB,Don Rehfeldt,Wisconsin,2,98,788,692,494,118,...,.758,20.2,7.1,5.0,1.2,3.5,.048,,,1950
3,4,TRI,Bob Cousy,Holy Cross,14,924,30165,16960,4786,6955,...,.803,35.3,18.4,5.2,7.5,91.1,.139,,,1950
4,5,WSC,Dick Schnittker,Ohio State,6,364,6744,3030,1372,479,...,.825,20.1,8.3,3.8,1.3,22.0,.132,,,1950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8005,56,TOR,Edin Bavčić,,,,,,,,...,,,,,,,,,,2006
8006,57,MIN,Loukas Mavrokefalidis,,,,,,,,...,,,,,,,,,,2006
8007,58,DAL,J.R. Pinnock,George Washington,,,,,,,...,,,,,,,,,,2006
8008,59,SAS,Damir Markota,,1,30,170,51,31,6,...,.636,5.7,1.7,1.0,0.2,-0.2,-.070,-8.5,-0.3,2006


In [6]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8010 entries, 0 to 8009
Data columns (total 22 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Pk       6980 non-null   object
 1   Tm       6976 non-null   object
 2   Player   6976 non-null   object
 3   College  6976 non-null   object
 4   Yrs      6976 non-null   object
 5   G        6976 non-null   object
 6   MP       6976 non-null   object
 7   PTS      6976 non-null   object
 8   TRB      6976 non-null   object
 9   AST      6976 non-null   object
 10  FG%      6976 non-null   object
 11  3P%      6976 non-null   object
 12  FT%      6976 non-null   object
 13  MP       6976 non-null   object
 14  PTS      6976 non-null   object
 15  TRB      6976 non-null   object
 16  AST      6976 non-null   object
 17  WS       6976 non-null   object
 18  WS/48    6976 non-null   object
 19  BPM      6976 non-null   object
 20  VORP     6976 non-null   object
 21  Year     8010 non-null   int64 
dtype

In [11]:
# cleaning empty rows
empty_or_zero = df_full[df_full['Pk'].isna() | (df_full['Pk'] == 0)]
empty_or_zero['Year']

12      1950
13      1950
26      1950
27      1950
40      1950
        ... 
7855    2004
7916    2005
7917    2005
7978    2006
7979    2006
Name: Year, Length: 1030, dtype: int64

In [ ]:
df_full = df_full.dropna(subset=['Pk'])

In [24]:
# reset indexes after cleaning NaNs
df_full = df_full.reset_index(drop=True)

In [61]:
new_columns = df_full.columns.tolist()

# fixing duplicate columns names

index_MP = new_columns.index('MP', new_columns.index('MP') + 1)
index_PTS = new_columns.index('PTS', new_columns.index('PTS') + 1)
index_TRB = new_columns.index('TRB', new_columns.index('TRB') + 1)
index_AST = new_columns.index('AST', new_columns.index('AST') + 1)

new_columns[index_MP] = 'MPperG'
new_columns[index_PTS] = 'PTSperG'
new_columns[index_TRB] = 'RBperG'
new_columns[index_AST] = 'ASTperG'

df_full.columns = new_columns


In [68]:
df_full.tail()

,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,AST,...,FT%,MPperG,PTSperG,RBperG,ASTperG,WS,WS/48,BPM,VORP,Year
6975,56,TOR,Edin Bavčić,,,,,,,,...,,,,,,,,,,2006
6976,57,MIN,Loukas Mavrokefalidis,,,,,,,,...,,,,,,,,,,2006
6977,58,DAL,J.R. Pinnock,George Washington,,,,,,,...,,,,,,,,,,2006
6978,59,SAS,Damir Markota,,1,30,170,51,31,6,...,.636,5.7,1.7,1.0,0.2,-0.2,-.070,-8.5,-0.3,2006
6979,60,DET,Will Blalock,Iowa State,1,14,166,25,15,17,...,1.000,11.9,1.8,1.1,1.2,-0.1,-.032,-5.8,-0.2,2006


In [69]:
df_full.columns

Index(['Pk', 'Tm', 'Player', 'College', 'Yrs', 'G', 'MP', 'PTS', 'TRB', 'AST',
       'FG%', '3P%', 'FT%', 'MPperG', 'PTSperG', 'RBperG', 'ASTperG', 'WS',
       'WS/48', 'BPM', 'VORP', 'Year'],
      dtype='object')

In [72]:
# changing MP to numeric
df_full['MP'] = pd.to_numeric(df_full['MP'], errors='coerce')
df_full['MP'] = df_full['MP'].fillna(0)

In [74]:
df_full.tail()

,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,AST,...,FT%,MPperG,PTSperG,RBperG,ASTperG,WS,WS/48,BPM,VORP,Year
6975,56,TOR,Edin Bavčić,,,,0.0,,,,...,,,,,,,,,,2006
6976,57,MIN,Loukas Mavrokefalidis,,,,0.0,,,,...,,,,,,,,,,2006
6977,58,DAL,J.R. Pinnock,George Washington,,,0.0,,,,...,,,,,,,,,,2006
6978,59,SAS,Damir Markota,,1,30,170.0,51,31,6,...,.636,5.7,1.7,1.0,0.2,-0.2,-.070,-8.5,-0.3,2006
6979,60,DET,Will Blalock,Iowa State,1,14,166.0,25,15,17,...,1.000,11.9,1.8,1.1,1.2,-0.1,-.032,-5.8,-0.2,2006


In [77]:
df_full.to_csv('nba_drafts_data.csv', index=False)